In [30]:
import cv2
import numpy as np
import os
from PIL import Image

In [31]:
def create_user(f_id, name):
    web=cv2.VideoCapture(0)
    web.set(3,640)
    web.set(4,480)

    faces= cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

    f_dir= 'dataset'
    f_name= name
    path=os.path.join(f_dir, f_name)
    if not os.path.isdir(path):
        os.makedirs(path)
    counter=0
    while(True):
        ret,img=web.read()
        img=cv2.flip(img,1)
        gray= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #facedetection and output is in the form [x,y,w,h]=x,y of top left corner of face rectangle. Identifies if any faces are present and their coordinate rectangles.
        multi_face=faces.detectMultiScale(gray, 1.3, 5)
        for x,y,w,h in multi_face:
            cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2)
            counter++1
            #storage also starts of the faces syntax= imwrite(path, image)
            cv2.imwrite("{}/{}.{}.{}{}".format(path, name, f_id, counter, ".jpg"), gray[y:y+h, x:x+w])
            cv2.imshow("Image", img)
        k=cv2.waitKey(30) & 0xff
        if k==27:
            break
        elif counter>=40:
            break
    web.release()
    cv2.destroyAllWindows() 

In [11]:
create_user(1,"suchet")

In [32]:
def train():
    database= 'dataset'
    img_dir=[x[0] for x in os.walk(database)][1::]
    recognizer= cv2.face.LBPHFaceRecognizer_create()
    detector= cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
    faceSamples=[]
    ids=[]
    for path in img_dir:
        path = str(path)
        imagepaths = [os.path.join(path, f) for f in os.listdir(path) if f.endswith(('.jpg', 'png', '.jpeg'))]

        for imagepath in imagepaths:
            PIL_img= Image.open(imagepath).convert('L')
            img_numpy= np.array(PIL_img, 'uint8')

            id=int(os.path.split(imagepath)[-1].split('.')[1])
            faces=detector.detectMultiScale(img_numpy)

            for (x,y,w,h) in faces:
                faceSamples.append(img_numpy[y:y+h, x:x+w])
                ids.append(id)
    recognizer.train(faceSamples, np.array(ids))
    recognizer.write('trainer.yml')
    print('\n[INFO] {0} faces trained.Existing Program'.format(len(np.unique(ids))))
    return len(np.unique(ids))

In [15]:
train()


[INFO] 1 faces trained.Existing Program


1

In [33]:
def recognize(names):
    recognizer= cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('trainer.yml')
    CascadePath= "haarcascade_frontalface_default.xml"
    faceCascade= cv2.CascadeClassifier(cv2.data.haarcascades+ CascadePath)
    font=cv2.FONT_HERSHEY_SIMPLEX
    id=0
    name=""
    face_count=0
    cam= cv2.VideoCapture(0)
    cam.set(3,640)
    cam.set(4,480)

    minW=0.1*cam.get(3)
    minH=0.1*cam.get(4)
    while True:
        ret, img= cam.read()
        img=cv2.flip(img,1)
        gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces=faceCascade.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=5,
            minSize=(int(minW), int(minH)),
            
        )
        for x,y,w,h in faces:
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
            id, confidence= recognizer.predict(gray[y:y+h, x:x+w])
            if(confidence<70):
                id=names[id]
            else:
                id="unknown"
                roll = None
                confidence= "{0}%".format(round(100-confidence))
                text= "I can't recognize You"
                #for attendence the code starts below

            if name==id:
                face_count+=1
                if(face_count>21):
                    fountcount=-100
            else:
                name=id
                face_count=0
                      
            cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
            cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)
            cv2.imshow("Image", img)
        k=cv2.waitKey(30) & 0xff
        if k==27:
            break
    cam.release()
    cv2.destroyAllWindows()

In [34]:
def run():
    giveinput={1:"Suchet"}
    nameinput=input("Enter Your Name")
    id_no=int(input("Serial no"))
    giveinput[id_no]=nameinput
    create_user(id_no, nameinput)
    train()
    recognize(giveinput)
    for key in giveinput:
        print(key, giveinput[key])

In [40]:
run()

Enter Your Name sakshi
Serial no 2



[INFO] 2 faces trained.Existing Program
1 Suchet
2 sakshi


In [41]:
run()

Enter Your Name sakshi
Serial no 2



[INFO] 2 faces trained.Existing Program
1 Suchet
2 sakshi
